In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import HTML
import scipy.optimize as opt
from scipy.stats import linregress
import plotly.express as px
import kaleido #required

In [32]:
# raw_xls = pd.read_excel(io='./verify_data/data2.xlsx')

raw = pd.read_table("./optimize/square.txt", sep="\s+")

raw_x = np.array(raw.iloc[0:, 0], dtype=np.float64)
raw_y = np.array(raw.iloc[0:, 1], dtype=np.float64)
raw_z = np.array(raw.iloc[0:, 2], dtype=np.float64)
raw_Bx = np.array(raw.iloc[0:, 3], dtype=np.float64)
raw_By = np.array(raw.iloc[0:, 4], dtype=np.float64)
raw_Bz = np.array(raw.iloc[0:, 5], dtype=np.float64)


# raw_Bx = np.abs(np.array(raw.iloc[0:, 3], dtype=np.float64) + 1j * np.array(raw.iloc[0:, 4], dtype=np.float64))
# raw_By = np.abs(np.array(raw.iloc[0:, 5], dtype=np.float64) + 1j * np.array(raw.iloc[0:, 6], dtype=np.float64))
# raw_Bz = np.abs(np.array(raw.iloc[0:, 7], dtype=np.float64) + 1j * np.array(raw.iloc[0:, 8], dtype=np.float64))

In [33]:
read_B_vect = np.column_stack((raw_Bx, raw_By, raw_Bz))
read_B_scalar = np.linalg.norm(read_B_vect, axis=1)

In [34]:
x_basis = np.unique(raw_x)
y_basis = np.unique(raw_y)
z_basis = np.unique(raw_z)

In [35]:
z_basis

array([0.00131])

In [36]:
read_B_vect

array([[ 7.32836965e-02,  7.09971005e-02,  5.80344495e-01],
       [ 9.25500528e-02,  1.06565653e-01,  7.34999789e-01],
       [ 1.11578824e-01,  1.58187016e-01,  9.24002562e-01],
       [ 1.23619713e-01,  2.27554244e-01,  1.13983968e+00],
       [ 1.18404298e-01,  3.08815099e-01,  1.35934239e+00],
       [ 8.74446676e-02,  3.84627747e-01,  1.54320506e+00],
       [ 3.24227154e-02,  4.31081243e-01,  1.64862947e+00],
       [-3.24227154e-02,  4.31081243e-01,  1.64862947e+00],
       [-8.74446676e-02,  3.84627747e-01,  1.54320506e+00],
       [-1.18404298e-01,  3.08815099e-01,  1.35934239e+00],
       [-1.23619713e-01,  2.27554244e-01,  1.13983968e+00],
       [-1.11578824e-01,  1.58187016e-01,  9.24002562e-01],
       [-9.25500528e-02,  1.06565653e-01,  7.34999789e-01],
       [-7.32836965e-02,  7.09971005e-02,  5.80344495e-01],
       [ 1.09694914e-01,  8.97385176e-02,  7.43540602e-01],
       [ 1.49985699e-01,  1.45930830e-01,  9.86561833e-01],
       [ 1.98554069e-01,  2.38415079e-01

In [37]:
# for CST values
# B_vect_raw = read_B_vect.reshape(len(np.unique(raw_z)), len(np.unique(raw_y)), len(np.unique(raw_x)), 3)
# B_scalar_raw = read_B_scalar.reshape(len(np.unique(raw_z)), len(np.unique(raw_y)), len(np.unique(raw_x)))

# for exported from solver

B_vect_raw = np.transpose(read_B_vect.reshape(len(np.unique(raw_y)), len(np.unique(raw_x)), len(np.unique(raw_z)), 3),
                          (1, 0, 2, 3))
B_scalar_raw = np.transpose(read_B_scalar.reshape(len(np.unique(raw_y)), len(np.unique(raw_x)), len(np.unique(raw_z))),
                            (1, 0, 2))

In [38]:
def plot_results_pltly_animate(mesh_basis, value):
    max_val = np.max(value[np.isfinite(value)])

    value[np.isinf(value)] = max_val

    # create a list of frames
    frames = []
    # create a frame for every line y
    if len(np.shape(value)) > 2:
        grad = go.Contour(x=mesh_basis[0], y=mesh_basis[1], z=value[:, :, 0], colorscale="turbo",
                          contours_coloring='heatmap')
        for i in range(np.shape(value)[2]):
            # update the line
            grad = go.Contour(x=mesh_basis[0], y=mesh_basis[1], z=value[:, :, i], colorscale="turbo",
                              contours_coloring='heatmap')
            # create the button
            button = {
                "type": "buttons",
                "buttons": [
                    {
                        "label": "Play",
                        "method": "animate",
                        "args": [None, {"frame": {"duration": 20}}],
                    }
                ],
            }
            # add the button to the layout and update the
            # title to show the gradient descent step
            layout = go.Layout(updatemenus=[button],
                               title_text=f"Gradient Descent Step {i}")
            # create a frame object
            frame = go.Frame(
                data=[grad],
                layout=go.Layout(title_text=f"Gradient Descent Step {i}")
            )
            # add the frame object to the frames list
            frames.append(frame)
        # combine the graph_objects into a figure
        fig = go.Figure(data=[grad],
                        frames=frames,
                        layout=layout)
    else:
        grad = go.Contour(x=mesh_basis[0], y=mesh_basis[1], z=value, colorscale="turbo", contours_coloring='heatmap')
        fig = go.Figure(data=[grad])

    # fig = go.Figure(data =
    # go.Contour(x=mesh_basis[0], y = mesh_basis[1], z=value, colorscale="turbo", contours_coloring='heatmap'))

    fig.update_layout(width=800,
                      height=800)

    fig.show(auto_play=True)

In [39]:
B_scalar = np.transpose(B_scalar_raw, (1, 0, 2))
B_vect = np.transpose(B_vect_raw, (1, 0, 2, 3))

plot_results_pltly_animate([x_basis, y_basis], B_scalar)
# plot_results_pltly_simple([x_basis, y_basis], B_vect[:,:,4,2])
# plot_results_pltly_simple([x_basis, y_basis], np.transpose(B_scalar, (0, 1, 2)))

In [40]:
def get_start_conditions(filename="start.txt"):
    points = []
    with open(filename, 'r') as file:
        for line in file.readlines():
            try:
                coords = [float(i.strip()) for i in line.split(",")]
            except:
                print("File incorrect!")
            points.append(coords)
    return np.array(points)

In [41]:
start_conditions = get_start_conditions("./optimize/square_start_conditions.txt")

In [42]:
start_conditions

array([[-0.0001, -0.005 ,  0.    ],
       [-0.005 , -0.005 ,  0.    ],
       [-0.005 ,  0.005 ,  0.    ],
       [ 0.005 ,  0.005 ,  0.    ],
       [ 0.005 , -0.005 ,  0.    ],
       [ 0.0001, -0.005 ,  0.    ]])

In [43]:
def plot_circuit(circuit, mesh_basis, value):
    x_circuit = np.array(circuit)[:, 0]
    y_circuit = np.array(circuit)[:, 1]
    z_circuit = np.array(circuit)[:, 2]

    max_val = np.max(value[np.isfinite(value)])

    value[np.isinf(value)] = max_val

    contour = go.Contour(x=mesh_basis[0], y=mesh_basis[1], z=value, colorscale="turbo", contours_coloring='heatmap')

    circuit_plot = go.Scatter(x=x_circuit, y=y_circuit, mode='lines',
                              line=dict(color='#fbfbfb', width=5))

    fig = make_subplots()
    fig.add_trace(contour)
    fig.add_trace(circuit_plot)
    fig.update_layout(width=800,
                      height=800, xaxis=dict(tickangle=90))

    fig.show()

In [44]:
from main import *

In [45]:
def get_solve(circuit, mesh_boundaries, mesh_points_count=None, mesh_step=None, flat=True):
    if mesh_points_count is None and mesh_step is None:
        raise "Mesh step or mesh points count in not defined!"
    # current = float(input("Current through the circuit, in amperes: "))
    # mesh_points_count = float(input("Mesh points count, in points per shortest axis: "))
    current = 1
    # if mesh_step is None:
    #     mesh_step, mesh_boundaries = mesh_step_calc(circuit, solver_box, mesh_points_count)
    # else:
    #     _, mesh_boundaries = mesh_step_calc(circuit, solver_box)
    # print("Mesh step ", mesh_step)
    mesh_basis = [np.arange(mesh_boundaries[i][0], mesh_boundaries[i][1] + mesh_step / 3, mesh_step) for i in
                  range(len(mesh_boundaries))]
    # if flat:
    #     mesh_basis[2] = [solver_box[2]]
    # mesh_basis = [i if len(i) > 0 else np.append(i, 0) for i in mesh_basis] # for the existence of at least one element
    # print(mesh_basis)
    mesh = generate_mesh(mesh_basis)
    circuit_segments = get_segments(circuit)
    vect_solve, magnitude_solve = mesh_solver(circuit_segments, mesh_basis, current)

    return circuit, mesh_step, mesh_basis, mesh, magnitude_solve, vect_solve

In [46]:
points_per_length = len(x_basis)
research_box = np.round(
    [[np.min(x_basis), np.max(x_basis)], [np.min(y_basis), np.max(y_basis)], [np.min(z_basis), np.max(z_basis)]], 10)
print(research_box)

[[-0.01703  0.01703]
 [-0.01703  0.01703]
 [ 0.00131  0.00131]]


In [47]:
def rsquared(x, y):
    """ Return R^2 where x and y are array-like."""
    try:
        slope, intercept, r_value, p_value, std_err = linregress(x, y)
    except:
        r_value = 0
    return r_value ** 2

In [48]:
def to_minimize(circuit_points, plot=False):
    circuit = circuit_points.reshape((len(circuit_points) // 3, 3))
    print(circuit)
    if len(circuit) == len(np.unique(circuit, axis=0)):
        _, _, _, _, magnitude_solve, vect_solve = get_solve(circuit, research_box,
                                                            mesh_step=round(np.abs(x_basis[0] - x_basis[1]), 9))
        difference = np.array([vect_solve, B_vect])
        difference_vector = np.linalg.norm(difference[0] - difference[1], axis=3)
        if plot:
            # plot_results_pltly_animate([x_basis, y_basis], difference_vector)
            plot_circuit(circuit, [x_basis, y_basis], difference_vector[:, :, 0])


        # metrics = np.std(np.ravel(difference_vector))
        metrics = np.max(difference_vector)
        # metrics = - rsquared(np.ravel(magnitude_solve), np.ravel(B_scalar))
        if np.isnan(metrics):
            metrics = np.inf
    else:
        metrics = np.inf
    print(metrics)
    return metrics


In [49]:
to_minimize(np.ravel(start_conditions), plot=True)

[[-0.0001 -0.005   0.    ]
 [-0.005  -0.005   0.    ]
 [-0.005   0.005   0.    ]
 [ 0.005   0.005   0.    ]
 [ 0.005  -0.005   0.    ]
 [ 0.0001 -0.005   0.    ]]


1.4210854715202004e-14


1.4210854715202004e-14

In [50]:
start_vector = np.ravel(start_conditions)
k = 3
plot_condition = np.concatenate((start_vector[:1], start_vector[1:15]*k, start_vector[15:16], start_vector[16:]*k)).reshape((len(start_vector) // 3, 3))
plot_circuit(plot_condition, [x_basis, y_basis], B_scalar[:, :, 0])

In [51]:
start_vector = np.ravel(start_conditions)

In [52]:
k_range = np.arange(0.1, 10, 0.1)
# print(k_range)
metric_range = list(map(lambda x: to_minimize(np.concatenate((start_vector[:1], start_vector[1:15]*x, start_vector[15:16], start_vector[16:]*x))), k_range))

[[-0.0001 -0.0005  0.    ]
 [-0.0005 -0.0005  0.    ]
 [-0.0005  0.0005  0.    ]
 [ 0.0005  0.0005  0.    ]
 [ 0.0005 -0.0005  0.    ]
 [ 0.0001 -0.0005  0.    ]]
129.9530640749734
[[-0.0001 -0.001   0.    ]
 [-0.001  -0.001   0.    ]
 [-0.001   0.001   0.    ]
 [ 0.001   0.001   0.    ]
 [ 0.001  -0.001   0.    ]
 [ 0.0001 -0.001   0.    ]]
130.30245309254954
[[-0.0001 -0.0015  0.    ]
 [-0.0015 -0.0015  0.    ]
 [-0.0015  0.0015  0.    ]
 [ 0.0015  0.0015  0.    ]
 [ 0.0015 -0.0015  0.    ]
 [ 0.0001 -0.0015  0.    ]]
130.86459589750598
[[-0.0001 -0.002   0.    ]
 [-0.002  -0.002   0.    ]
 [-0.002   0.002   0.    ]
 [ 0.002   0.002   0.    ]
 [ 0.002  -0.002   0.    ]
 [ 0.0001 -0.002   0.    ]]
131.52028494124346
[[-0.0001 -0.0025  0.    ]
 [-0.0025 -0.0025  0.    ]
 [-0.0025  0.0025  0.    ]
 [ 0.0025  0.0025  0.    ]
 [ 0.0025 -0.0025  0.    ]
 [ 0.0001 -0.0025  0.    ]]
131.7853896269844
[[-0.0001 -0.003   0.    ]
 [-0.003  -0.003   0.    ]
 [-0.003   0.003   0.    ]
 [ 0.003   

In [53]:
fig = px.line(x=k_range, y=metric_range, title='Size relation for MAX metric')
fig.update_layout(
    height=800, width=1000, xaxis_title="K-factor", yaxis_title="MAX H difference, A/m",
    font=dict(
        size=18
    )
)
fig.show()
fig.write_image('./plots/max_k.png')

In [54]:
offset_matrix = lambda x: np.array([[x, 0, 0] for _ in range(len(start_conditions))])
start_vector_offset = lambda x: np.ravel(start_conditions+offset_matrix(x))
delta = 0.012
plot_condition = start_vector_offset(delta).reshape((len(start_vector) // 3, 3))
plot_circuit(plot_condition, [x_basis, y_basis], B_scalar[:, :, 0])

In [55]:
start_vector = np.ravel(start_conditions)
off_range = np.arange(-0.012, 0.012, 0.0002)
offset_matrix = lambda x: np.array([[x, 0, 0] for _ in range(len(start_conditions))])
start_vector_offset = lambda x: np.ravel(start_conditions+offset_matrix(x))
metric_range_offset = list(map(lambda x: to_minimize(start_vector_offset(x)), off_range))

[[-0.0121 -0.005   0.    ]
 [-0.017  -0.005   0.    ]
 [-0.017   0.005   0.    ]
 [-0.007   0.005   0.    ]
 [-0.007  -0.005   0.    ]
 [-0.0119 -0.005   0.    ]]
151.15007702059555
[[-0.0119 -0.005   0.    ]
 [-0.0168 -0.005   0.    ]
 [-0.0168  0.005   0.    ]
 [-0.0068  0.005   0.    ]
 [-0.0068 -0.005   0.    ]
 [-0.0117 -0.005   0.    ]]
162.84750272969282
[[-0.0117 -0.005   0.    ]
 [-0.0166 -0.005   0.    ]
 [-0.0166  0.005   0.    ]
 [-0.0066  0.005   0.    ]
 [-0.0066 -0.005   0.    ]
 [-0.0115 -0.005   0.    ]]
172.34790150968996
[[-0.0115 -0.005   0.    ]
 [-0.0164 -0.005   0.    ]
 [-0.0164  0.005   0.    ]
 [-0.0064  0.005   0.    ]
 [-0.0064 -0.005   0.    ]
 [-0.0113 -0.005   0.    ]]
178.66953915080276
[[-0.0113 -0.005   0.    ]
 [-0.0162 -0.005   0.    ]
 [-0.0162  0.005   0.    ]
 [-0.0062  0.005   0.    ]
 [-0.0062 -0.005   0.    ]
 [-0.0111 -0.005   0.    ]]
181.46943694647834
[[-0.0111 -0.005   0.    ]
 [-0.016  -0.005   0.    ]
 [-0.016   0.005   0.    ]
 [-0.006 

In [56]:
fig = px.line(x=off_range, y=metric_range_offset, title='Offset relation for MAX metric')
fig.update_layout(
    height=800 , xaxis_title="Offset, m", yaxis_title="MAX H difference, A/m",
    font=dict(
        size=18
    )
)
fig.show()
fig.write_image('./plots/max_off.png')

In [57]:
to_minimize(start_vector_offset(0.012), plot=True)

[[ 0.0119 -0.005   0.    ]
 [ 0.007  -0.005   0.    ]
 [ 0.007   0.005   0.    ]
 [ 0.017   0.005   0.    ]
 [ 0.017  -0.005   0.    ]
 [ 0.0121 -0.005   0.    ]]


151.15007702059555


151.15007702059555